In [5]:
n=3

In [7]:
G = SymmetricGroup(n)

In [7]:
SGA = G.algebra(QQ)

In [8]:
A = SGA.dft(); A

[   1    1    1    1    1    1]
[   1 -1/2 -1/2  1/2  1/2   -1]
[   0 -3/4  3/4  3/4 -3/4    0]
[   0    1   -1    1   -1    0]
[   1 -1/2 -1/2 -1/2 -1/2    1]
[   1    1    1   -1   -1   -1]

In [93]:
#is the matrix unitary?
#A*A^T, the Gram matrix, is diagonal, but not the identity. so A is not unitary.

In [9]:
print((A*A.transpose()).str())

[  6   0   0   0   0   0]
[  0   3   0   0   0   0]
[  0   0 9/4   0   0   0]
[  0   0   0   4   0   0]
[  0   0   0   0   3   0]
[  0   0   0   0   0   6]


In [1]:
#define a new DFT which is unitary
#NOTE: in Beals' ['97] he normalizes by \sqrt{d_\lambda/n!}
#but also notes that a basis change is an equivalence relation on rep'ns
#and each equivalence class contains a unitary representatione
#if each rep'n \rho \in \hat{G} is unitary, then the transformation is unitary
#these representations are not unitary
#to make them unitary, use Weyl's unitary trick: 
#define a new inner product <x,y>_\rho = (1/|G|)\sum_{g \in G}<\rho(g)x,\rho(g)y>
#compute an orthonormal basis using this inner product

In [177]:
#define the Fourier coefficient at the representation specht_module
#which is the Specht module corresponding to partition
def hat(f,partition):
    n = sum(partition)
    G = SymmetricGroup(n)
    specht_module = G.algebra(QQ).specht_module(partition)
    d_part = specht_module.dimension()
    return sqrt(d_part/G.order())*sum(f(g)*specht_module.representation_matrix(g) for g in G)

In [178]:
#define the delta function delta_s(t) = {1 if s == t, 0 otherwise}
def delta(s):
    return lambda t: 1 if t == s else 0

In [179]:
#for each basis element g \in G
#compute the Fourier coefficients \hat{\delta_g}(partition) for all partitions
#make the Fourier coefficients into a list via .list()
#flatten the list of lists to get a vector \hat{g}
#put the vectors into a list and take the transpose to get the DFT
from sage.misc.flatten import flatten
def unitary_dft(n):
    return matrix([flatten([hat(delta(g),partition).list() for partition in Partitions(n)]) for g in G]).transpose()

In [378]:
#define the G-averaged inner product given a representation
#<x,y>_\rho = (1/|G|)\sum_{g \in G}<\rho(g)x,\rho(g)y>
#where x, y are coordinate vectors for elements in V_\rho
def inv_inner_product(partition,x,y):
    n = sum(partition)
    G = SymmetricGroup(n)
    rho = G.algebra(QQ).specht_module(partition).representation_matrix
    return (1/G.order())*sum((rho(g)*vector(x))*(rho(g)*vector(y)) for g in G)

In [379]:
#define the projection of the vector v onto vector u
def proj(u,v):
    return ((vector(v)*vector(u))/(vector(u)*vector(u)))*vector(u)

In [380]:
#define Gram-Schmidt orthonormalization process that works in the symbolic ring
#assume A = [v_1, v_2, ... , v_k] and each v_j is a column vector
#NOTE: GS, M = A.gram_schmidt(orthonormal=True) works for some rings, but not \QQ or exact rings
def gram_schmidt_SR(A,partition,orthonormal=False):
    v = A.transpose()
    u = []
    e = []
    for k in range(A.ncols()):
        u_k = v[k] - sum(proj(u[j],v[k]) for j in range(k))
        u.append(u_k)
        e.append(u_k/(u_k*u_k))
    if orthonormal:
        return matrix(e).transpose()
    else:
        return matrix(u).transpose()

In [374]:
#find the change-of-basis matrix M for which A = M*GS where GS is the Gram-Schmidt orthonormal basis of A
def unitary_change_of_basis(A,partition):
    orthonormal_vecs = gram_schmidt_SR(A,partition,orthonormal=True)
    return A*orthonormal_vecs.inverse()

In [311]:
#define the inner product matrix A = [<e_i,e_j>_\rho] for some ordered basis of {e_i} of V_\rho
def gram_matrix(partition,base_ring=QQ):
    specht_module = G.algebra(QQ).specht_module(partition)
    d_part = specht_module.dimension()
    e = lambda i: [1 if i==j else 0 for j in range(d_part)]
    return matrix(base_ring,[[inv_inner_product(partition,e(i),e(j)) for j in range(d_part)] for i in range(d_part)])

In [354]:
#ISSUE: the change-of-basis matrix is not making the representations unitary
#DESCRIPTION: using the built-in method over the CDF ring, the matrix M gives the correct change-of-basis to give a unitary matrix. 
#now I just need to implement Gram-Schmidt properly to go from the Gram matrix to M, the change-of-basis matrix

In [61]:
partition = Partitions(n)[1]

In [62]:
specht_module = G.algebra(QQ).specht_module(partition)

In [131]:
rho = specht_module.representation_matrix

In [105]:
d_part = specht_module.dimension()

In [361]:
A = gram_matrix(partition)

In [362]:
A

[ 4/3 -2/3]
[-2/3  4/3]

In [365]:
M = unitary_change_of_basis(A,partition); M

[ 68/45 -64/45]
[-16/45  68/45]

In [375]:
orthonormal_vecs = gram_schmidt_SR(A,partition,orthonormal=True); orthonormal_vecs

[  3/5   1/2]
[-3/10     1]

In [376]:
#check that inner products of columns are really zero
orthonormal_vecs.transpose()[0]*orthonormal_vecs.transpose()[1]

0

In [369]:
#check that \rho(g) is unitary w.r.t. invariant inner product
x = vector([1,2]); y = vector([2,3])
all(inv_inner_product(partition,rho(g)*x,rho(g)*y) == inv_inner_product(partition,x,y) for g in G)

True

In [370]:
g = G[4]

In [371]:
U = M.inverse()*rho(g)*M; U*U.transpose()

[ 949/1125 1088/1125]
[1088/1125 2581/1125]

In [372]:
(U*x).inner_product(U*y)

72/25

In [373]:
x.inner_product(y)

8

In [262]:
identity_matrix(d_part) == M*orthonormal_vecs

True

In [382]:
#note that the built-in method is working. I must be computing M wrong via Gram-Schmidt
A_2 = gram_matrix(partition,QQbar)
GS_2, M_2 = A_2.gram_schmidt(orthonormal=True); M_2

[ 1.490711984999860?                   0]
[-1.192569587999888? 0.8944271909999159?]

In [352]:
U_2 = M_2.inverse()*rho(g)*M_2; U_2*U_2.transpose()

[    0.9999999999999999 2.6201263381153693e-16]
[2.6201263381153693e-16     1.0000000000000002]

In [180]:
#check that the DFT is unitary
unitary_dft(3)*unitary_dft(3).transpose()

[   1    0    0    0    0    0]
[   0  4/3 -2/3  2/3 -1/3    0]
[   0 -2/3  4/3 -1/3  2/3    0]
[   0  2/3 -1/3  4/3 -2/3    0]
[   0 -1/3  2/3 -2/3  4/3    0]
[   0    0    0    0    0    1]

In [58]:
#what are the eigenvalues?
#n=3: two real, two complex
#n=4: all complex
#the magnitude is not 1, they're closely grouped around 2 or 3

In [92]:
eigs = A.eigenvalues(); eigs

[-1.927382440195288?, 2.121715010353017?, -1.174423554578382? - 1.623003052198150?*I, -1.174423554578382? + 1.623003052198150?*I, 1.452257269499517? - 1.086817272076326?*I, 1.452257269499517? + 1.086817272076326?*I]

In [93]:
[abs(eig) for eig in eigs]

[1.927382440195288?,
 2.121715010353017?,
 2.003349593304431?,
 2.003349593304431?,
 1.813897174510622?,
 1.813897174510622?]

In [121]:
#note that the singluar values are the square roots of the diagonal entries of the Gram matrix
print(SymmetricGroup(n).algebra(CDF).dft().SVD()[1].numpy().diagonal())
print(sqrt((A*A.transpose()).numpy().diagonal()))

[2.44948974+0.j 2.44948974+0.j 2.        +0.j 1.73205081+0.j
 1.73205081+0.j 1.5       +0.j]
[2.44948974 1.73205081 1.5        2.         1.73205081 2.44948974]
